In [1]:
import parl
from parl import layers
import paddle.fluid as fluid
import copy
import numpy as np
import os
# import gym
from parl.utils import logger

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
class Model(parl.Model):
    def __init__(self, act_dim): # input 64 * 39
        hid1_size = 64
        #self.conv1 = layers.conv2d(layers.conv2d(num_filters=8, filter_size=3, act='relu')
        self.fc1 = layers.fc(size=hid1_size, act = 'relu')
        self.fc2 = layers.fc(size=hid1_size, act = "relu")
        self.fc3 = layers.fc(size=act_dim)

    def value(self, obs): # 正向传播
        out = self.fc1(obs)
        out = self.fc2(out)
        Q = self.fc3(out)
        return Q

In [15]:
class DQN(parl.Algorithm):
    def __init__(self, model, act_dim=None, gamma=None, lr=None):
        """ DQN algorithm
        
        Args:
            model (parl.Model): 定义Q函数的前向网络结构
            act_dim (int): action空间的维度，即有几个action
            gamma (float): reward的衰减因子
            lr (float): learning rate 学习率.
        """
        self.model = model
        self.target_model = copy.deepcopy(model) 

        assert isinstance(act_dim, int)
        assert isinstance(gamma, float)
        assert isinstance(lr, float)
        self.act_dim = act_dim
        self.gamma = gamma
        self.lr = lr

    def predict(self, obs):
        """ 使用self.model的value网络来获取 [Q(s,a1),Q(s,a2),...]
        """
        return self.model.value(obs)

    def learn(self, obs, action, reward, next_obs, terminal): 
        """ 使用DQN算法更新self.model的value网络
        更新方法: 计算一组target_Q与pred_Q的MSE作为损失函数用梯度下降更新network, 而不是Qtable方法中的单步公式更新
        """
        # 从target_model中获取 max Q' 的值，用于计算target_Q
        next_pred_value = self.target_model.value(next_obs)
        best_v = layers.reduce_max(next_pred_value, dim=1) # the largest Q value
        best_v.stop_gradient = True  # 阻止梯度传递
        terminal = layers.cast(terminal, dtype='float32') # 转成浮点数
        target = reward + (1.0 - terminal) * self.gamma * best_v # 若是最后一步, 则target = reward, 否则 =reward +  gamma * best_v

        pred_value = self.model.value(obs)  # 获取Q预测值
        # 将action转onehot向量，比如：3 => [0,0,0,1,0] 
        action_onehot = layers.one_hot(action, self.act_dim)
        action_onehot = layers.cast(action_onehot, dtype='float32')
        # 下面一行是逐元素相乘，拿到action对应的 Q(s,a)
        # 比如：pred_value = [[2.3, 5.7, 1.2, 3.9, 1.4]], action_onehot = [[0,0,0,1,0]]
        #  ==> pred_action_value = [[3.9]]
        pred_action_value = layers.reduce_sum(
            layers.elementwise_mul(action_onehot, pred_value), dim=1)

        # 计算 Q(s,a) 与 target_Q的均方差，得到loss
        cost = layers.square_error_cost(pred_action_value, target)
        cost = layers.reduce_mean(cost)
        optimizer = fluid.optimizer.Adam(learning_rate=self.lr)  # 使用Adam优化器
        optimizer.minimize(cost)
        
        self.lr *= 0.998
        self.lr = max(self.lr,1e-6)

        return cost

    def sync_target(self):
        """ 把 self.model 的模型参数值同步到 self.target_model
        """
        self.model.sync_weights_to(self.target_model)

In [16]:
class Agent(parl.Agent):
    def __init__(self,
                 algorithm,
                 obs_dim,
                 act_dim,
                 e_greed=0.1,
                 e_greed_decrement=0):
        assert isinstance(obs_dim, int)
        assert isinstance(act_dim, int)
        self.obs_dim = obs_dim 
        self.act_dim = act_dim
        super(Agent, self).__init__(algorithm)

        self.global_step = 0
        self.update_target_steps = 400  # 每隔200个training steps再把model的参数复制到target_model中

        self.e_greed = e_greed  # 有一定概率随机选取动作，探索
        self.e_greed_decrement = e_greed_decrement  # 随着训练逐步收敛，探索的程度慢慢降低

    def build_program(self): # 相当于与alg进行数据传输
        self.pred_program = fluid.Program()
        self.learn_program = fluid.Program()

        with fluid.program_guard(self.pred_program):  # 搭建计算图用于 预测动作，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            self.value = self.alg.predict(obs)

        with fluid.program_guard(self.learn_program):  # 搭建计算图用于 更新Q网络，定义输入输出变量
            obs = layers.data(
                name='obs', shape=[self.obs_dim], dtype='float32')
            action = layers.data(name='act', shape=[1], dtype='int32')
            reward = layers.data(name='reward', shape=[], dtype='float32')
            next_obs = layers.data(
                name='next_obs', shape=[self.obs_dim], dtype='float32')
            terminal = layers.data(name='terminal', shape=[], dtype='bool')
            self.cost = self.alg.learn(obs, action, reward, next_obs, terminal)

    def sample(self, obs):
        sample = np.random.rand()  # 产生0~1之间的小数
        if sample < self.e_greed:
            act = np.random.randint(self.act_dim)  # 探索：每个动作都有概率被选择
        else:
            act = self.predict(obs)  # 选择最优动作
        self.e_greed = max(
            1e-7, self.e_greed - self.e_greed_decrement)  # 随着训练逐步收敛，探索的程度慢慢降低
        return act

    def predict(self, obs):  # 选择最优动作
        obs = np.expand_dims(obs, axis=0)
        pred_Q = self.fluid_executor.run(
            self.pred_program,
            feed={'obs': obs.astype('float32')},
            fetch_list=[self.value])[0]
        pred_Q = np.squeeze(pred_Q, axis=0)
        act = np.argmax(pred_Q)  # 选择Q最大的下标，即对应的动作
        return act

    def learn(self, obs, act, reward, next_obs, terminal):
        # 每隔200个training steps同步一次model和target_model的参数
        if self.global_step % self.update_target_steps == 0:
            self.alg.sync_target()
        self.global_step += 1

        act = np.expand_dims(act, -1)
        feed = {
            'obs': obs.astype('float32'),
            'act': act.astype('int32'),
            'reward': reward,
            'next_obs': next_obs.astype('float32'),
            'terminal': terminal
        }
        cost = self.fluid_executor.run(
            self.learn_program, feed=feed, fetch_list=[self.cost])[0]  # 训练一次网络
        return cost

In [17]:
import random
import collections
import numpy as np

class ReplayMemory(object):
    def __init__(self, max_size):
        self.buffer = collections.deque(maxlen=max_size) # 队列

    # 增加'一条'经验到经验池中
    def append(self, exp):
        self.buffer.append(exp)

    # 从经验池中选取N条经验出来
    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        obs_batch, action_batch, reward_batch, next_obs_batch, done_batch = [], [], [], [], []

        for experience in mini_batch:
            s, a, r, s_p, done = experience
            obs_batch.append(s)
            action_batch.append(a)
            reward_batch.append(r)
            next_obs_batch.append(s_p)
            done_batch.append(done)

        return np.array(obs_batch).astype('float32'), \
            np.array(action_batch).astype('float32'), np.array(reward_batch).astype('float32'),\
            np.array(next_obs_batch).astype('float32'), np.array(done_batch).astype('float32')

    def __len__(self):
        return len(self.buffer)

In [40]:
def get_obs(p):
    obs = np.array(list(p.getGameState().values()))
    obs[3:5] -= obs[0]
    obs[6:8] -= obs[0]    
    obs = (obs/np.array([512, 10, 512, 256, 256, 512, 256, 256]))-0.5
    return obs

# 训练一个episode
def run_episode(p, agent, rpm): #pygame
    total_reward = 0
    p.reset_game()
    obs = get_obs(p)
    act_space = p.getActionSet()
    
    step = 0; train_loss=[]
    while True:
        step += 1
        action = agent.sample(obs)  # 采样动作，所有动作都有概率被尝试到
        #next_obs, reward, done, _ = env.step(action)  
        
        reward = p.act(act_space[action]) 
        next_obs = get_obs(p)
        done = p.game_over()
        if reward>0:
            reward = 5.0*reward
        rpm.append((obs, action, reward, next_obs, done)) # SARS    

        # train model
        if (len(rpm) > MEMORY_WARMUP_SIZE) and (step % LEARN_FREQ == 0):
            (batch_obs, batch_action, batch_reward, batch_next_obs,
             batch_done) = rpm.sample(BATCH_SIZE)
            train_loss.append( agent.learn(batch_obs, batch_action, batch_reward,
                                     batch_next_obs,
                                     batch_done ))  # s,a,r,s',done

        total_reward += reward
        obs = next_obs
        if done:
            loss = np.mean(train_loss)
            break
    return total_reward, loss


# 评估 agent, 跑 5 个episode，总reward求平均
# 评估 agent, 跑 5 个episode，总reward求平均
def evaluate(p, agent):
    eval_reward = []
    act_space = p.getActionSet()
    
    for i in range(10):
        p.reset_game()
        obs = get_obs(p)
        episode_reward = 0
        while True:
            action = agent.predict(obs)  # 预测动作，只选最优动作
            reward = p.act(act_space[action])
            next_obs = get_obs(p)
            done = p.game_over()
            episode_reward += reward
            obs = next_obs
            if done:
                break
        eval_reward.append(episode_reward)
        print("Test{}: ".format(i+1), episode_reward)
    return np.mean(eval_reward)

### Hyper Parameters

In [20]:
LEARN_FREQ = 5 # 训练频率，不需要每一个step都learn，攒一些新增经验后再learn，提高效率
MEMORY_SIZE = 2000    # replay memory的大小，越大越占用内存
MEMORY_WARMUP_SIZE = 200  # replay_memory 里需要预存一些经验数据，再开启训练
BATCH_SIZE = 16   # 每次给agent learn的数据数量，从replay memory随机里sample一批数据出来
LEARNING_RATE = 1e-4 # 学习率
GAMMA = 0.90 # reward 的衰减因子，一般取 0.9 到 0.999 不等

e_greed=1e-4

### Initialise

In [ ]:
from ple.games.flappybird import FlappyBird
game = FlappyBird()
from ple import PLE

p = PLE(game, fps=30, display_screen=False, force_fps=False)
p.init()

action_dim = len(p.getActionSet())
obs_shape = 8


# 根据parl框架构建agent
model = Model(act_dim=action_dim)
algorithm = DQN(model, act_dim=action_dim, gamma=GAMMA, lr=LEARNING_RATE)
agent = Agent(
    algorithm,
    obs_dim=obs_shape,
    act_dim=action_dim,
    e_greed=e_greed,  # 有一定概率随机选取动作，探索
    e_greed_decrement=5e-7)  # 随着训练逐步收敛，探索的程度慢慢降低

rpm = ReplayMemory(MEMORY_SIZE)  # DQN的经验回放池

while len(rpm) < MEMORY_WARMUP_SIZE:
    run_episode(p, agent, rpm)

agent.restore("_.ckpt")

### Training

In [ ]:
p = PLE(game, fps=30, display_screen=False, force_fps=False)
p.init()
episode = 0
max_episode = 200

while episode < max_episode:  # 训练max_episode个回合，test部分不计算入episode数量
    # train part
    losses = []
    for i in range(0, 50):
        reward, loss = run_episode(p, agent, rpm)
        losses.append(loss)
        episode += 1
        print(int(reward), end='|')

    # test part
    eval_reward = evaluate(p, agent)  # render=True 查看显示效果
    print('\n')
    
    logger.info('episode:{}  e_greed:{}  test_reward:{}  mean_loss:{}'.format(
        episode, np.round(agent.e_greed, 3), eval_reward, np.mean(losses)))

    # 保存模型
    if eval_reward>= 0:
        save_path = './flappybird_3fc64_ep{}_r{}_.ckpt'.format(episode,eval_reward)
        agent.save(save_path)

In [38]:
# save_path = './flappybird_model_128.ckpt'.format(max_episode - episode)
# agent.save(save_path)

### Visualised Testing

In [ ]:
p = PLE(game, fps=30, display_screen=False, force_fps=False)
p.init()

agent.restore("./flappybird_3fc64.ckpt")
p.reset_game()
evaluate(p, agent)